# Parse tables to database

https://nbviewer.jupyter.org/github/FinanceData/OpenDartReader/blob/master/docs/OpenDartReader_reference_manual.ipynb


```
KOSPI
└── Companys
    └── Receipts
        ├── BS: Financial statement(Balance Sheet)
        ├── IS: Income Statement
        ├── CIS: Comprehensive Income Statement
        ├── CF: Cash flow statement
        └── SCE: Statement of Changes in Equity
```

<p align="center">
    <img alt="Alt Text" src="https://g.gravizo.com/svg?digraph%20G%20%7B%0A%20%201%20%5Blabel%3D%22KOSPI%22%2C%20fontcolor%3Dblack%2C%20shape%3Dbox%5D%3B%0A%20%202%20%5Blabel%3D%22Company1%22%2C%20fontcolor%3Dblack%2C%20shape%3Dbox%5D%3B%0A%20%203%20%5Blabel%3D%22Company2%22%2C%20fontcolor%3Dblack%2C%20shape%3Dbox%5D%3B%0A%20%204%20%5Blabel%3D%22...%22%2C%20fontcolor%3Dblack%2C%20shape%3Dbox%5D%3B%0A%20%205%20%5Blabel%3D%22Receipt%20No.1%22%2C%20fontcolor%3Dblack%2C%20shape%3Dbox%5D%3B%0A%20%206%20%5Blabel%3D%22Receipt%20No.2%22%2C%20fontcolor%3Dblack%2C%20shape%3Dbox%5D%3B%20%20%20%20%0A%20%207%20%5Blabel%3D%22...%22%2C%20fontcolor%3Dblack%2C%20shape%3Dbox%5D%3B%0A%20%208%20%5Blabel%3D%22BS%22%2C%20fontcolor%3Dblack%2C%20shape%3Dbox%5D%3B%0A%20%209%20%5Blabel%3D%22IS%22%2C%20fontcolor%3Dblack%2C%20shape%3Dbox%5D%3B%0A%20%2010%20%5Blabel%3D%22CIS%22%2C%20fontcolor%3Dblack%2C%20shape%3Dbox%5D%3B%0A%20%2011%20%5Blabel%3D%22CF%22%2C%20fontcolor%3Dblack%2C%20shape%3Dbox%5D%3B%0A%20%2012%20%5Blabel%3D%22SCE%22%2C%20fontcolor%3Dblack%2C%20shape%3Dbox%5D%3B%0A%20%201%20-%3E%202%20-%3E%205%3B%0A%20%201%20-%3E%203%3B%0A%20%201%20-%3E%204%3B%0A%20%202%20-%3E%206%3B%0A%20%202%20-%3E%207%3B%0A%20%206%20-%3E%208%3B%0A%20%206%20-%3E%209%3B%0A%20%206%20-%3E%2010%3B%0A%20%206%20-%3E%2011%3B%0A%20%206%20-%3E%2012%3B%0A%7D" />
</p>
<details>
<summary>How to create graph in markdown?</summary>

```python
from urllib.parse import quote
raw = """digraph G {
  1 [label="KOSPI", fontcolor=black, shape=box];
  2 [label="Company1", fontcolor=black, shape=box];
  3 [label="Company2", fontcolor=black, shape=box];
  4 [label="...", fontcolor=black, shape=box];
  5 [label="Receipt No.1", fontcolor=black, shape=box];
  6 [label="Receipt No.2", fontcolor=black, shape=box];    
  7 [label="...", fontcolor=black, shape=box];
  8 [label="BS", fontcolor=black, shape=box];
  9 [label="IS", fontcolor=black, shape=box];
  10 [label="CIS", fontcolor=black, shape=box];
  11 [label="CF", fontcolor=black, shape=box];
  12 [label="SCE", fontcolor=black, shape=box];
  1 -> 2 -> 5;
  1 -> 3;
  1 -> 4;
  2 -> 6;
  2 -> 7;
  6 -> 8;
  6 -> 9;
  6 -> 10;
  6 -> 11;
  6 -> 12;
}"""
txt = quote(raw)
```
    
copy the text behind https://g.gravizo.com/svg?
</details>

# Finance DataReader

In [1]:
from private.apikey import APIKEY

import pandas as pd
import OpenDartReader
import FinanceDataReader as fdr

dart = OpenDartReader(APIKEY)

In [2]:
stocks = fdr.StockListing("KOSPI")
stocks = stocks.loc[~stocks["Sector"].isnull(), :]
stocks_syms = stocks["Symbol"].values
stocks = stocks.reset_index(drop=True)

In [3]:
for c in stocks.columns:
    print(c, stocks[c].dtype)

Symbol object
Market object
Name object
Sector object
Industry object
ListingDate datetime64[ns]
SettleMonth object
Representative object
HomePage object
Region object


In [4]:
stocks["ListingDate"] = stocks["ListingDate"].astype(str)

In [5]:
import sqlite3
from pathlib import Path

db_path = Path("./private/")
conn = sqlite3.connect(db_path / "kospi_new.db")
# Samsung Electronic: 005930
# conn = sqlite3.connect(db_path / "samsung_new.db")
c = conn.cursor()

In [6]:
stocks.to_sql("kospi", conn, index=True)

## Company Basic

- corp_name: 정식명칭
- corp_name_eng: 영문명칭
- stock_name: 종목명 또는 약식명칭 
- stock_code: 상장회사인 경우 주식의 종목코드
- ceo_nm: 대표자명
- crop_cls: 법인구분
- jurir_no: 법인등록번호
- bizr_no: 사업자등록번호
- adres: 주소
- hm_url: 홈페이지
- ir_url: IR홈페이지
- phn_no: 전화번호
- fax_no: 팩스번호
- induty_code: 업종코드
- estdt: 설립일
- acc_mt: 결산월

In [7]:
from tqdm.notebook import tqdm

In [8]:
basic_cols = [
    'corp_name', 'corp_name_eng', 'stock_name', 'stock_code', 'ceo_nm', 'corp_cls', 
    'jurir_no', 'bizr_no', 'adres', 'hm_url', 'ir_url', 'phn_no', 'fax_no', 'induty_code', 'est_dt', 'acc_mt'
]
df_basic = pd.DataFrame([dart.company(s) for s in tqdm(stocks_syms)]).loc[:, basic_cols]
df_basic.to_sql("company", conn, index=True)

  0%|          | 0/816 [00:00<?, ?it/s]

In [9]:
for col in df_basic.columns:
    print(col, df_basic[col].dtype)

corp_name object
corp_name_eng object
stock_name object
stock_code object
ceo_nm object
corp_cls object
jurir_no object
bizr_no object
adres object
hm_url object
ir_url object
phn_no object
fax_no object
induty_code object
est_dt object
acc_mt object


In [10]:
# df_samsung = pd.DataFrame([dart.company("005930")]).loc[:, basic_cols]

# DROP TABLE if exists
# sql = "DROP TABLE company"
# res = c.execute(sql)

# insert into "company" table
# df_samsung.to_sql("company", conn, index=True)

In [11]:
sql = "SELECT * FROM company"
res = c.execute(sql)
col_company = list(map(lambda x: x[0], res.description))
for col, value in zip(col_company, res.fetchone()):
    print(f"{col}: {value}")

index: 0
corp_name: AJ네트웍스 주식회사
corp_name_eng: AJ Networks Co.,Ltd.
stock_name: AJ네트웍스
stock_code: 095570
ceo_nm: 박대현
corp_cls: Y
jurir_no: 1101111874654
bizr_no: 2148648586
adres: 서울특별시 송파구 정의로8길 9 (문정동,AJ빌딩)
hm_url: www.ajnet.co.kr
ir_url: 
phn_no: 02-6363-9999
fax_no: 02-6240-0888
induty_code: 76320
est_dt: 20000210
acc_mt: 12


## Report

- corp (문자열): 검색대상 회사의 종목코드를 지정합니다. 고유번호, 회사이름도 가능합니다.
- key_word (문자열): 조회 내용 지정, 아래 "key_word 항목"을 참고하십시오 ('증자','배당','자기주식','최대주주','최대주주변동','소액주주','임원','직원','임원개인보수','임원전체보수','개인별보수','타법인출자')
- bsns_year (문자열 혹은 정수값): 사업연도
- reprt_code (문자열): 보고서 코드 ('11013'=1분기보고서, '11012'=반기보고서, '11014'=3분기보고서, '11011'=사업보고서)

반환값 (DataFrame): 조회 결과를 데이터프레임(DataFrame)으로 반환합니다. 데이터프레임의 각 컬럼은 다음과 같습니다.

- rcept_no: 접수번호
- corp_cls: 법인구분 Y(유가), K(코스닥), N(코넥스), E(기타)
- corp_code: 고유번호
- corp_name: 법인명

key_word 항목 지정에 따라 결과 데이터의 컬럼이 달라집니다. '배당' - 배당에 관한 사항

- se: 구분. 유상증자(주주배정), 전환권행사 등
- stock_knd: 주식 종류
- thstrm: 당기
- frmtrm: 전기
- lwfr: 전전기

In [141]:
corp = "005930"
bsns_year = 2020

In [13]:
# ['증자', '배당', '자기주식', '최대주주', '최대주주변동', '소액주주', '임원', '직원', '임원개인보수', '임원전체보수', '개인별보수', '타법인출자']
dart.report(corp, '개인별보수', bsns_year, reprt_code='11011')

,rcept_no,corp_cls,corp_code,corp_name,nm,ofcps,mendng_totamt,mendng_totamt_ct_incls_mendng
0,20210309000744,Y,00126380,삼성전자,권오현,고 문,"17,233,000,000",-
1,20210309000744,Y,00126380,삼성전자,윤부근,고 문,"11,527,000,000",-
2,20210309000744,Y,00126380,삼성전자,신종균,고 문,"11,327,000,000",-
3,20210309000744,Y,00126380,삼성전자,전동수,前고문,"10,908,000,000",-
4,20210309000744,Y,00126380,삼성전자,김기남,대표이사,"8,274,000,000",-


## Finstate All

인자

- corp (문자열): 검색대상 회사의 종목코드를 지정합니다. 고유번호, 회사이름도 가능합니다.
- bsns_year (문자열 혹은 정수값): 사업연도
- reprt_code (문자열): 보고서 코드 ('11013'=1분기보고서, '11012' =반기보고서, '11014'=3분기보고서, '11011'=사업보고서)
- fs_div (문자열): 재무제표가 재무제표인지 ('CFS'=연결제무제표, 'OFS'=별도(개별)제무제표)

필요한 계정 항목

|sj_div|sj_nm|account_nm
|---|---|---
|BS|재무상태표|유동자산
|BS|재무상태표|현금및현금성자산
|BS|재무상태표|매출채권
|BS|재무상태표|선급비용
|BS|재무상태표|재고자산
|BS|재무상태표|비유동자산
|BS|재무상태표|유형자산
|BS|재무상태표|무형자산
|BS|재무상태표|자산총계
|BS|재무상태표|유동부채
|BS|재무상태표|매입채무
|BS|재무상태표|단기차입금
|BS|재무상태표|선수금
|BS|재무상태표|비유동부채
|BS|재무상태표|사채
|BS|재무상태표|장기차입금
|BS|재무상태표|부채총계
|BS|재무상태표|자본총계
|BS|재무상태표|부채와자본총계
|IS|손익계산서|수익(매출액)
|IS|손익계산서|매출원가
|IS|손익계산서|매출총이익
|IS|손익계산서|판매비와관리비
|IS|손익계산서|영업이익
|IS|손익계산서|금융수익
|IS|손익계산서|금융비용
|IS|손익계산서|법인세비용차감전순이익(손실)
|IS|손익계산서|법인세비용
|IS|손익계산서|계속영업이익(손실)
|IS|손익계산서|당기순이익(손실)
|CF|현금흐름표|영업활동 현금흐름
|CF|현금흐름표|영업에서 창출된 현금흐름
|CF|현금흐름표|투자활동 현금흐름
|CF|현금흐름표|재무활동 현금흐름
|CF|현금흐름표|외화환산으로 인한 현금의 변동
|CF|현금흐름표|현금및현금성자산의 순증감
|CF|현금흐름표|기초의 현금및현금성자산
|CF|현금흐름표|기말의 현금및현금성자산

In [142]:
# accounts = [
#     ('BS', '유동자산'),('BS', '현금및현금성자산'),('BS', '매출채권'),('BS', '선급비용'),('BS', '재고자산'),('BS', '비유동자산'),('BS', '유형자산'),('BS', '무형자산'),('BS', '자산총계'),('BS', '유동부채'),('BS', '매입채무'),
#     ('BS', '단기차입금'),('BS', '선수금'),('BS', '비유동부채'),('BS', '사채'),('BS', '장기차입금'),('BS', '부채총계'),('BS', '자본총계'),('BS', '부채와자본총계'),('IS', '수익(매출액)'),('IS', '매출원가'),('IS', '매출총이익'),
#     ('IS', '판매비와관리비'),('IS', '영업이익'),('IS', '금융수익'),('IS', '금융비용'),('IS', '법인세비용차감전순이익(손실)'),('IS', '법인세비용'),('IS', '계속영업이익(손실)'),('IS', '당기순이익(손실)'),('CF', '영업활동 현금흐름'),
#     ('CF', '영업에서 창출된 현금흐름'),('CF', '투자활동 현금흐름'),('CF', '재무활동 현금흐름'),('CF', '외화환산으로 인한 현금의 변동'),('CF', '현금및현금성자산의 순증감'),('CF', '기초의 현금및현금성자산'),('CF', '기말의 현금및현금성자산')
# ]
# _, account_nms = list(zip(*accounts))
cols = ['rcept_no', 'reprt_code', 'bsns_year', 'corp_code', 'sj_div', 'sj_nm', 'account_id', 'account_nm', 'thstrm_nm','thstrm_amount']
# sj_divs = ['BS', 'IS', 'CF']

# change some names
def change_account_values(x):
    if x == "수익(매출액)":
        return '매출액'
    else:
        return x.replace("이익(손실)", "손익")

def get_accounts(comprehensive=False):
    if comprehensive:
        accounts = [
            ('BS', '유동자산'),('BS', '현금및현금성자산'),('BS', '매출채권'),('BS', '선급비용'),('BS', '재고자산'),('BS', '비유동자산'),('BS', '유형자산'),('BS', '무형자산'),('BS', '자산총계'),('BS', '유동부채'),('BS', '매입채무'),
            ('BS', '단기차입금'),('BS', '선수금'),('BS', '비유동부채'),('BS', '사채'),('BS', '장기차입금'),('BS', '부채총계'),('BS', '자본총계'),('BS', '부채와자본총계'),('CIS', '수익(매출액)'),('CIS', '매출원가'),('CIS', '매출총이익'),
            ('CIS', '판매비와관리비'),('CIS', '영업이익'),('CIS', '금융수익'),('CIS', '금융비용'),('CIS', '법인세비용차감전순이익(손실)'),('CIS', '법인세비용'),('CIS', '계속영업이익(손실)'),('CIS', '당기순이익(손실)'),('CF', '영업활동 현금흐름'),
            ('CF', '영업에서 창출된 현금흐름'),('CF', '투자활동 현금흐름'),('CF', '재무활동 현금흐름'),('CF', '외화환산으로 인한 현금의 변동'),('CF', '현금및현금성자산의 순증감'),('CF', '기초의 현금및현금성자산'),('CF', '기말의 현금및현금성자산')
        ]
    else:
        accounts = [
            ('BS', '유동자산'),('BS', '현금및현금성자산'),('BS', '매출채권'),('BS', '선급비용'),('BS', '재고자산'),('BS', '비유동자산'),('BS', '유형자산'),('BS', '무형자산'),('BS', '자산총계'),('BS', '유동부채'),('BS', '매입채무'),
            ('BS', '단기차입금'),('BS', '선수금'),('BS', '비유동부채'),('BS', '사채'),('BS', '장기차입금'),('BS', '부채총계'),('BS', '자본총계'),('BS', '부채와자본총계'),('IS', '수익(매출액)'),('IS', '매출원가'),('IS', '매출총이익'),
            ('IS', '판매비와관리비'),('IS', '영업이익'),('IS', '금융수익'),('IS', '금융비용'),('IS', '법인세비용차감전순이익(손실)'),('IS', '법인세비용'),('IS', '계속영업이익(손실)'),('IS', '당기순이익(손실)'),('CF', '영업활동 현금흐름'),
            ('CF', '영업에서 창출된 현금흐름'),('CF', '투자활동 현금흐름'),('CF', '재무활동 현금흐름'),('CF', '외화환산으로 인한 현금의 변동'),('CF', '현금및현금성자산의 순증감'),('CF', '기초의 현금및현금성자산'),('CF', '기말의 현금및현금성자산')
        ]
    return accounts
    
def process_accounts(df, cols, comprehensive=False):
    accounts = get_accounts(comprehensive=comprehensive)
    idx = [i for i, x in df.loc[:, ['sj_div', 'account_nm']].iterrows() if tuple(x.values) in accounts]
    # assert len(idx) == len(accounts), f"account number is not right: {len(idx)}"
    df = df.loc[idx, cols].reset_index(drop=True)
    df['account_nm'] = df.loc[:, 'account_nm'].apply(change_account_values)
    return df

In [143]:
df_samsung = dart.finstate_all(corp, bsns_year, reprt_code='11011', fs_div='CFS')
df_samsung = process_accounts(df_samsung, cols, comprehensive=False)

In [144]:
df_samsung

,rcept_no,reprt_code,bsns_year,corp_code,sj_div,sj_nm,account_id,account_nm,thstrm_nm,thstrm_amount
0,20210309000744,11011,2020,00126380,BS,재무상태표,ifrs-full_CurrentAssets,유동자산,제 52 기,198215579000000
1,20210309000744,11011,2020,00126380,BS,재무상태표,ifrs-full_CashAndCashEquivalents,현금및현금성자산,제 52 기,29382578000000
2,20210309000744,11011,2020,00126380,BS,재무상태표,dart_ShortTermTradeReceivable,매출채권,제 52 기,30965058000000
3,20210309000744,11011,2020,00126380,BS,재무상태표,-표준계정코드 미사용-,선급비용,제 52 기,2266100000000
4,20210309000744,11011,2020,00126380,BS,재무상태표,ifrs-full_Inventories,재고자산,제 52 기,32043145000000
5,20210309000744,11011,2020,00126380,BS,재무상태표,ifrs-full_NoncurrentAssets,비유동자산,제 52 기,180020139000000
6,20210309000744,11011,2020,00126380,BS,재무상태표,ifrs-full_PropertyPlantAndEquipment,유형자산,제 52 기,128952892000000
7,20210309000744,11011,2020,00126380,BS,재무상태표,ifrs-full_IntangibleAssetsOtherThanGoodwill,무형자산,제 52 기,18468502000000
8,20210309000744,11011,2020,00126380,BS,재무상태표,ifrs-full_Assets,자산총계,제 52 기,378235718000000
9,20210309000744,11011,2020,00126380,BS,재무상태표,ifrs-full_CurrentLiabilities,유동부채,제 52 기,75604351000000


## Finstate
- corp (문자열): 검색대상 회사의 종목코드를 지정합니다. 고유번호, 회사이름도 가능합니다.
- bsns_year (문자열 혹은 정수값): 사업연도
- reprt_code (문자열): 보고서 코드 ('11013'=1분기보고서, '11012'=반기보고서, '11014'=3분기보고서, '11011'=사업보고서)

반환값 (DataFrame): 조회 결과를 데이터프레임(DataFrame)으로 반환합니다. 데이터프레임의 각 컬럼은 다음과 같습니다.

- rcept_no: 접수번호
- corp_code: 사업 연도
- stock_code: 종목 코드
- reprt_code: 보고서 코드
- account_nm: 계정명 (예: 자본총계)
- fs_div: 개별/연결구분 ('CFS'=연결재무제표, 'OFS'=재무제표)
- fs_nm: 개별/연결명 ('연결재무제표' 또는 '재무제표')
- sj_div: 재무제표구분 ('BS'=재무상태표, 'IS'=손익계산서)
- sj_nm: 재무제표명 ( '재무상태표' 또는 '손익계산서')
- thstrm_nm: 당기명
- thstrm_dt: 당기일자
- thstrm_amount: 당기금액
- thstrm_add_amount: 당기누적금액
- frmtrm_nm: 전기명
- frmtrm_dt: 전기일자
- frmtrm_amount: 전기금액
- frmtrm_add_amount: 전기누적금액
- bfefrmtrm_nm: 전전기명
- bfefrmtrm_dt: 전전일자
- bfefrmtrm_amount: 전전기금액
- ord: 계정과목 정렬순서

In [38]:
# df_summary = dart.finstate(corp, bsns_year, reprt_code="11011")
# for col in ["thstrm_amount", "frmtrm_amount", "bfefrmtrm_amount"]:
#     df_summary.loc[:, col] = df_summary.loc[:, col].apply(lambda x: int("".join(x.split(","))))
    
# cols = ["fs_div", "sj_div", "account_nm", "thstrm_nm", "thstrm_amount"] # "frmtrm_nm", "frmtrm_amount", "bfefrmtrm_nm", "bfefrmtrm_amount"]
# df_summary.loc[:, cols].groupby(["thstrm_nm", "fs_div", "sj_div", "account_nm"]).agg("sum")

## Query 와 SQL 만들기

- [시간]: BS: '제\*\*기', '20\*\*년도', '올해', '작년' / IS: '20\*\*년도 부터 20\*\*년 까지', 
- [계정]: BS: '유동자산', '비유동자산', '자산총계', '유동부채', '비유동부채', '부채총계', '자본금', '이익잉여금', '자본총계' / IS: '매출액', '영업이익', '법인세차감전 순이익', '당기순이익'
- [질문]: '얼마나돼', '얼마야', '어떻게돼'

**format의 형태**

틀린 질문을 만들수 없게 로직짜기

단순 계정 관련 질문:
- Q: 삼성전자 [시간]의 [계정]은/는 [질문]? 
- Q: [시간]에서 삼성전자의 [계정]은/는 [질문]?

## XBRL 표준계정과목체계(계정과목)

In [26]:
# def preprocess_finstate(df):
#     for col in ["thstrm_amount", "frmtrm_amount", "bfefrmtrm_amount"]:
#         if col in df.columns:
#             df.loc[:, col] = df.loc[:, col].apply(lambda x: int("".join(x.split(","))))
            
#     return df

# corp = "005930"
# bsns_years = list(range(2015, 2021))

# df_receipts = pd.concat([preprocess_finstate(dart.finstate(corp, bsns_year, reprt_code="11011")).iloc[:12, :-1] for bsns_year in bsns_years]).reset_index(drop=True)
# df_receipts.loc[:, "bsns_year"] = df_receipts.loc[:, "bsns_year"].astype(int)

## receipts all

당기 만 저장

In [110]:
import time
from collections import defaultdict

In [128]:
bsns_years = list(range(2018, 2021))
dfs = defaultdict(list)
cs = defaultdict(list)
for corp in tqdm(stocks_syms):
#     dfs[corp] = []
    for bsns_year in bsns_years:
        df = dart.finstate_all(corp, bsns_year, reprt_code='11011', fs_div='CFS')
        
        time.sleep(1)
        if df is None:
            continue
        else:
            comprehensive = True if df['sj_div'].isin(['IS']).sum() == 0 else False
            cs[corp].append((bsns_year, comprehensive))
            df = process_accounts(df, cols, comprehensive=comprehensive)
            df.loc[df["thstrm_amount"].isin(["-"]), "thstrm_amount"] = "0"
            df.loc[:, "thstrm_amount"] = df.loc[:, "thstrm_amount"].apply(lambda x: int("".join(x.split(",")))).astype(int)
            df.loc[:, "bsns_year"] = df.loc[:, "bsns_year"].astype(int)
            dfs[corp].append(df)

  0%|          | 0/816 [00:00<?, ?it/s]

AssertionError: account number is not right: 24

In [131]:
corp, bsns_year, comprehensive

('095570', 2018, True)

In [133]:
accounts = get_accounts(comprehensive=comprehensive)
idx = [i for i, x in df.loc[:, ['sj_div', 'account_nm']].iterrows() if tuple(x.values) in accounts]

In [134]:
df.loc[idx]

,rcept_no,reprt_code,bsns_year,corp_code,sj_div,sj_nm,account_id,account_nm,account_detail,thstrm_nm,thstrm_amount,frmtrm_nm,frmtrm_amount,bfefrmtrm_nm,bfefrmtrm_amount,ord,thstrm_add_amount
0,20190531001150,11011,2018,00365387,BS,재무상태표,ifrs_CurrentAssets,유동자산,-,제 19 기,1511355997808,제 18 기,318877073839,제 17 기,345416728536,1,NaN
1,20190531001150,11011,2018,00365387,BS,재무상태표,ifrs_CashAndCashEquivalents,현금및현금성자산,-,제 19 기,53509862897,제 18 기,77402142843,제 17 기,128499184276,2,NaN
5,20190531001150,11011,2018,00365387,BS,재무상태표,ifrs_Inventories,재고자산,-,제 19 기,51456822550,제 18 기,54488684360,제 17 기,48758273799,6,NaN
9,20190531001150,11011,2018,00365387,BS,재무상태표,ifrs_NoncurrentAssets,비유동자산,-,제 19 기,1063783463551,제 18 기,2035356660661,제 17 기,1735900124893,10,NaN
16,20190531001150,11011,2018,00365387,BS,재무상태표,ifrs_PropertyPlantAndEquipment,유형자산,-,제 19 기,245055468592,제 18 기,332064561991,제 17 기,252633766155,17,NaN
17,20190531001150,11011,2018,00365387,BS,재무상태표,ifrs_IntangibleAssetsOtherThanGoodwill,무형자산,-,제 19 기,79481799795,제 18 기,69978715694,제 17 기,55569122687,18,NaN
22,20190531001150,11011,2018,00365387,BS,재무상태표,ifrs_Assets,자산총계,-,제 19 기,2575139461359,제 18 기,2354233734500,제 17 기,2081316853429,23,NaN
23,20190531001150,11011,2018,00365387,BS,재무상태표,ifrs_CurrentLiabilities,유동부채,-,제 19 기,1609070172105,제 18 기,1207742892817,제 17 기,992364518455,25,NaN
25,20190531001150,11011,2018,00365387,BS,재무상태표,ifrs_ShorttermBorrowings,단기차입금,-,제 19 기,257683028520,제 18 기,491426376541,제 17 기,397709232557,27,NaN
32,20190531001150,11011,2018,00365387,BS,재무상태표,ifrs_NoncurrentLiabilities,비유동부채,-,제 19 기,474261316507,제 18 기,699317960406,제 17 기,661466633151,34,NaN


In [163]:
account_nms = defaultdict(set)

for corp in tqdm(stocks_syms):
    for bsns_year in bsns_years:
        df = dart.finstate_all(corp, bsns_year, reprt_code='11011', fs_div='CFS')
        time.sleep(1)
        if df is None:
            continue
        else:
            sj_divs = ['BS', 'CIS', 'CF'] if df['sj_div'].isin(['IS']).sum() == 0 else ['BS', 'IS', 'CF']
            account_nms[corp] = account_nms[corp].union(df.loc[df['sj_div'].isin(sj_divs), 'account_nm'].tolist())

  0%|          | 0/816 [00:00<?, ?it/s]

In [165]:
import pickle

with open("./private/account_nms.pickle", 'wb') as file:
    pickle.dump(account_nms, file)

In [30]:
# 59 companys don's have receipts for now
sum([True if len(dfs[corp]) == 0 else False for corp in tqdm(stocks_syms)])

  0%|          | 0/810 [00:00<?, ?it/s]

60

In [31]:
# save to sql
for corp in tqdm(stocks_syms):
    if len(dfs[corp]) == 0:
        continue
    df = pd.concat(dfs[corp]).reset_index(drop=True)
    table_name = f"{corp}"
    df.to_sql(table_name, conn, index=True)

  0%|          | 0/810 [00:00<?, ?it/s]